In [328]:
%load_ext autoreload
%autoreload 2

from amd_bkg_matchids import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. load matches

In [234]:
with open('matches.json') as f:
    matches = json.load(f)
print(len(matches))

4461


# 2. Load Booking data

In [293]:
print("Loading Booking hotels")
bkgh = load_booking(matches.values())
print(len(bkgh))

Loading Booking hotels
4461


# 3. Inspect chain matches

In [ ]:
amdh = load_amadeus_from_db()

In [246]:
# Don't compute them again: chain_matches.json has been edited with manually checked matches
# get_chain_matches(amdh, bkgh, matches)

{u'AADX PLUS': defaultdict(int, {'NONE': 4}),
 u'ACCOR HOTELS': defaultdict(int,
             {u'Adagio Access Aparthotels': 43,
              u'Adagio Aparthotels': 47,
              u'Courtyard by Marriott': 1,
              u'Mercure': 1,
              'NONE': 3,
              u'Novotel': 1,
              u'Wyndham Hotels &amp; Resorts, Nobile Hoteis': 1,
              u'ibis': 1}),
 u'ALESIA DISTRIBUTION': defaultdict(int, {'NONE': 3}),
 u'AMADEUS LINKHOTEL': defaultdict(int,
             {u'Civitel Hotels &amp; Resorts': 1, 'NONE': 14}),
 u'ATEL HOTELS NETWORK': defaultdict(int, {'NONE': 5}),
 u'BAYMONT INNS': defaultdict(int,
             {u'Aloft': 1,
              u'Baymont Inn &amp; Suites': 320,
              u'Clarion': 1,
              u'Comfort Inn': 1,
              u'Comfort Suites': 1,
              u'Country Inns &amp; Suites': 2,
              u'Days Inn': 9,
              u'Drury': 1,
              u'Econo Lodge': 2,
              u'Fairfield Inn': 2,
              u

In [295]:
amdh.bkg_id = amdh.amd_id.apply(lambda i: matches[i] if i in matches else None)
matches_df = amdh.join(bkgh, on='bkg_id', how='inner', rsuffix='_bkg')
matches_df = matches_df[['amd_id', 'bkg_id_bkg',u'name', 'name_bkg', u'chain', 'chain_bkg', 'lat',  u'lng', u'lat_bkg', u'lng_bkg']]
mdf = matches_df
mdf.chain_bkg.fillna('', inplace=True)
len(matches_df)

4461

In [ ]:
# Explore suspicious chain matches
chain_a = "RESIDENCE INN HOTELS"
chain_b = "Marriott Hotels & Resorts"

filter_a = (mdf.chain == chain_a) if chain_a else (mdf.chain.isnull())
filter_b = (mdf.chain_bkg == chain_b) if chain_b else (mdf.chain_bkg.isnull())
mdf[filter_a & filter_b]

# 4. clean up erroneous chain matches

In [355]:
chain_matches = load_commented_json('chain_matches_checked.json')

In [356]:
sc = mdf.copy() # candidates to suspicious

sc.name = sc.name.apply(normalize)
sc.name_bkg = sc.name_bkg.apply(normalize)
sc.chain = sc.chain.astype(unicode).apply(normalize)
sc.chain_bkg = sc.chain_bkg.astype(unicode).apply(normalize)

filter_neq_name = sc.name != sc.name_bkg
sc = sc[filter_neq_name]

len(sc)


3596

In [359]:
suspicious = pd.DataFrame(data=[],columns=mdf.columns)

for chain_a, d in chain_matches.items():
    for chain_b in d:
        chain_a = normalize(chain_a) 
        chain_b = normalize(chain_b)
        filter_a = (sc.chain == chain_a) if chain_a != 'none' else (sc.chain.isnull())
        filter_b = (sc.chain_bkg == chain_b) if chain_b != 'none' else (sc.chain_bkg.isnull())
        filter_b = filter_b & ~(sc.name_bkg.str.contains(chain_a))
        filter_a = filter_a & ~(sc.name.str.contains(chain_b))
            
        suspicious = suspicious.append(sc[filter_a & filter_b])

len(suspicious)

214

In [346]:
suspicious.sample(50)

,amd_id,bkg_id_bkg,name,name_bkg,chain,chain_bkg,lat,lng,lat_bkg,lng_bkg
a1a3cad9-bd15-47ee-adb8-7be0b3ea9db2,a1a3cad9-bd15-47ee-adb8-7be0b3ea9db2,50398.0,appart blanc mesnil,campanile blanc mesnil,synxis,campanile,48.95008,2.46541,48.950424,2.466173
d3902cd8-9d35-4323-be5a-403ddac6b670,d3902cd8-9d35-4323-be5a-403ddac6b670,522029.0,home2 ste huntsville research,westin huntsville,home2 ste,westin,34.71945,-86.67564,34.719135,-86.672803
e0721769-243a-4705-a948-76d1ff608684,e0721769-243a-4705-a948-76d1ff608684,457344.0,woodspring ste jackson sout,red lion inn ste byram jackson south,sceptre hospitality,red lion hotels,32.19003,-90.25310,32.184598,-90.254278
38a27b21-8c5f-4875-a772-066adb3581ee,38a27b21-8c5f-4875-a772-066adb3581ee,371068.0,value place brownsville,6 brownsville,sceptre hospitality,motel6,25.94397,-97.50478,25.947993,-97.505990
c858681d-84b6-425a-87d3-da5f7714c919,c858681d-84b6-425a-87d3-da5f7714c919,270910.0,appart mpl millenaire,lagrange aparthotel montpellier millenaire,synxis,lagrange,43.61891,3.91905,43.610456,3.917539
ebcd0c32-99a0-4fd1-b898-01feefa4fd74,ebcd0c32-99a0-4fd1-b898-01feefa4fd74,490332.0,inn madison marriott,econo madison heights,inn hotels,econo,42.53175,-83.11989,42.532023,-83.120736
869692c9-3fb3-4c52-bae0-8ca152e8711e,869692c9-3fb3-4c52-bae0-8ca152e8711e,180713.0,inn galleri marriott,westin galleria,inn hotels,westin,29.74190,-95.46473,29.738133,-95.464363
afed086e-a419-44c1-a81d-2f91d8393b5d,afed086e-a419-44c1-a81d-2f91d8393b5d,184151.0,value place columbus southeast,holiday inn ste columbus southeast,sceptre hospitality,holiday inn,39.89394,-82.88269,39.891546,-82.881632
88387ef7-2f85-4136-aca2-4b24d276a5b2,88387ef7-2f85-4136-aca2-4b24d276a5b2,325128.0,value place montgomery,super 8 montgomery,sceptre hospitality,super 8,32.36153,-86.22166,32.361298,-86.217247
4a7155c1-1a68-4861-86b6-4558d9c13998,4a7155c1-1a68-4861-86b6-4558d9c13998,679745.0,gateway apart boroko,gateway,synxis,coral sea hotels,-9.44692,147.21437,-9.447111,147.213655


In [360]:
suspicious.to_excel('suspicious_matches.xls', index=False)

# 5. remove suspicious matches and save to public matches file

In [361]:
len(matches)

4461

In [364]:
clean_matches = {k:v for k,v in matches.items() if k not in suspicious.amd_id.values}

len(clean_matches)

4247

In [366]:
# percentage matched
len(clean_matches) * 1.0 / len(amdh)

0.8360236220472441

In [367]:
json_dump_unicode(clean_matches,'web/matches.json')